# 了解 PyTorch 缓冲区

本质上，PyTorch 缓冲区是与 PyTorch 模块或模型相关的张量属性，类似于参数，但与参数不同的是，缓冲区在训练期间不会更新。
PyTorch 中的缓冲区在处理 GPU 计算时特别有用，因为它们需要与模型参数一起在设备之间传输（例如从 CPU 到 GPU）。与参数不同，缓冲区不需要梯度计算，但它们仍然需要位于正确的设备上以确保所有计算正确执行。
在第 3 章中，我们通过`self.register_buffer`使用 PyTorch 缓冲区，书中仅对此进行了简要说明。由于概念和目的尚不清楚，因此此代码笔记本通过实践示例提供了更长的解释。

## 没有缓冲区的示例

假设我们有以下代码，它基于第 3 章中的代码。该版本已修改为排除缓冲区。它实现了 LLM 中使用的因果自注意力机制：

In [5]:
import torch
import torch.nn as nn

class CausalAttentionWithoutBuffers(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

我们可以在一些示例数据上初始化并运行该模块，如下所示：

In [6]:
torch.manual_seed(123)

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

batch = torch.stack((inputs, inputs), dim=0)
context_length = batch.shape[1]
d_in = inputs.shape[1]
d_out = 2

ca_without_buffer = CausalAttentionWithoutBuffers(d_in, d_out, context_length, 0.0)

with torch.no_grad():
    context_vecs = ca_without_buffer(batch)

print(context_vecs)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]])


到目前为止，一切都进展顺利。

然而，在训练 LLM 时，我们通常使用 GPU 来加速该过程。因此，我们将`CausalAttentionWithoutBuffers`模块转移到 GPU 设备上。

请注意，此操作需要代码在配备GPU的环境中运行。

In [7]:
has_cuda = torch.cuda.is_available()
has_mps = torch.backends.mps.is_available()

print("Machine has GPU:", has_cuda or has_mps)

if has_mps:
    device = torch.device("mps")   # Apple Silicon GPU (Metal)
elif has_cuda:
    device = torch.device("cuda")  # NVIDIA GPU
else:
    device = torch.device("cpu")   # CPU fallback

print(f"Using device: {device}")

batch = batch.to(device)
ca_without_buffer = ca_without_buffer.to(device)

Machine has GPU: True
Using device: cuda


现在，让我们再次运行代码：

In [8]:
with torch.no_grad():
    context_vecs = ca_without_buffer(batch)

print(context_vecs)

RuntimeError: expected self and mask to be on the same device, but got mask on cpu and self on cuda:0

运行代码导致错误。发生了什么？看起来我们尝试了 GPU 上的张量和 CPU 上的张量之间的矩阵乘法。但我们把模块移到了 GPU 上！？

让我们仔细检查一些张量的设备位置：

In [18]:
print("W_query.device:", ca_without_buffer.W_query.weight.device)
print("mask.device:", ca_without_buffer.mask.device)

W_query.device: cuda:0
mask.device: cpu


In [19]:
print("type of ca_without_buffer.W_query.weight: ",type(ca_without_buffer.W_query.weight))
print("type of ca_without_buffer.mask : ",type(ca_without_buffer.mask))

type of ca_without_buffer.W_query.weight:  <class 'torch.nn.parameter.Parameter'>
type of ca_without_buffer.mask :  <class 'torch.Tensor'>


正如我们所看到的，`mask`没有移到 GPU 上。这是因为它不是像权重那样的 PyTorch 参数（例如`W_query.weight`）。

这意味着我们必须通过 `.to("cuda")` 手动将其移动到 GPU：

In [20]:
ca_without_buffer.mask = ca_without_buffer.mask.to(device)
print("mask.device:", ca_without_buffer.mask.device)

mask.device: cuda:0


In [21]:
with torch.no_grad():
    context_vecs = ca_without_buffer(batch)

print(context_vecs)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], device='cuda:0')


这一次，成功了！

然而，记住将各个张量移动到 GPU 可能很乏味。正如我们将在下一节中看到的，使用`register_buffer`将`mask`注册为缓冲区会更容易。

## 一个带有缓冲区的例子

现在让我们修改因果注意类以将因果`mask`注册为缓冲区：

In [22]:
import torch
import torch.nn as nn

class CausalAttentionWithBuffer(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        # 旧:
        # self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

        # 新:
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

现在，方便的是，如果我们将模块移动到 GPU，掩码也将位于 GPU 上：

In [23]:
ca_with_buffer = CausalAttentionWithBuffer(d_in, d_out, context_length, 0.0)
ca_with_buffer.to(device)

print("W_query.device:", ca_with_buffer.W_query.weight.device)
print("mask.device:", ca_with_buffer.mask.device)

W_query.device: cuda:0
mask.device: cuda:0


In [24]:
with torch.no_grad():
    context_vecs = ca_with_buffer(batch)

print(context_vecs)

tensor([[[-0.5684,  0.5063],
         [-0.5388,  0.6447],
         [-0.5242,  0.6954],
         [-0.4578,  0.6471],
         [-0.4006,  0.5921],
         [-0.3997,  0.5971]],

        [[-0.5684,  0.5063],
         [-0.5388,  0.6447],
         [-0.5242,  0.6954],
         [-0.4578,  0.6471],
         [-0.4006,  0.5921],
         [-0.3997,  0.5971]]], device='cuda:0')


正如我们在上面所看到的，将张量注册为缓冲区可以让我们的方式变得更加轻松：我们不必记住手动将张量移动到 GPU 等目标设备。

## 缓冲区和 `state_dict`(`状态字典`)

- 与常规张量相比，PyTorch 缓冲区的另一个优点是它们包含在模型的`state_dict`中
- 例如，考虑没有缓冲区的因果注意对象的`state_dict`

In [25]:
ca_without_buffer.state_dict()

OrderedDict([('W_query.weight',
              tensor([[-0.2354,  0.0191, -0.2867],
                      [ 0.2177, -0.4919,  0.4232]], device='cuda:0')),
             ('W_key.weight',
              tensor([[-0.4196, -0.4590, -0.3648],
                      [ 0.2615, -0.2133,  0.2161]], device='cuda:0')),
             ('W_value.weight',
              tensor([[-0.4900, -0.3503, -0.2120],
                      [-0.1135, -0.4404,  0.3780]], device='cuda:0'))])

- 掩码不包含在上面的`state_dict`中
- 然而，由于将其注册为缓冲区，掩码*包含在下面的`state_dict`中

In [26]:
ca_with_buffer.state_dict()

OrderedDict([('mask',
              tensor([[0., 1., 1., 1., 1., 1.],
                      [0., 0., 1., 1., 1., 1.],
                      [0., 0., 0., 1., 1., 1.],
                      [0., 0., 0., 0., 1., 1.],
                      [0., 0., 0., 0., 0., 1.],
                      [0., 0., 0., 0., 0., 0.]], device='cuda:0')),
             ('W_query.weight',
              tensor([[-0.0746, -0.5366, -0.3570],
                      [ 0.4928,  0.0345, -0.4677]], device='cuda:0')),
             ('W_key.weight',
              tensor([[ 0.0911,  0.4770, -0.5456],
                      [-0.3887, -0.2299,  0.0232]], device='cuda:0')),
             ('W_value.weight',
              tensor([[-0.1347, -0.0634, -0.5629],
                      [ 0.2704,  0.5068,  0.3529]], device='cuda:0'))])

- 例如，在保存和加载经过训练的 PyTorch 模型时，`state_dict`非常有用
- 在这种特殊情况下，保存和加载`mask`可能不是非常有用，因为它在训练期间保持不变；因此，出于演示目的，我们假设它已被修改，其中所有`1`都更改为`2`：

In [27]:
ca_with_buffer.mask[ca_with_buffer.mask ==1.] =2.
ca_with_buffer.mask

tensor([[0., 2., 2., 2., 2., 2.],
        [0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 2., 2., 2.],
        [0., 0., 0., 0., 2., 2.],
        [0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0.]], device='cuda:0')

- 然后，如果我们保存并加载模型，我们可以看到掩模已恢复为修改后的值

In [28]:
torch.save(ca_with_buffer.state_dict(), "model.pth")

new_ca_with_buffer = CausalAttentionWithBuffer(d_in, d_out, context_length, 0.0)
new_ca_with_buffer.load_state_dict(torch.load("model.pth"))

new_ca_with_buffer.mask

tensor([[0., 2., 2., 2., 2., 2.],
        [0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 2., 2., 2.],
        [0., 0., 0., 0., 2., 2.],
        [0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0.]])

- 如果我们不使用缓冲区，则事实并非如此：

In [29]:
ca_without_buffer.mask[ca_without_buffer.mask == 1.] = 2.

torch.save(ca_without_buffer.state_dict(), "model.pth")

new_ca_without_buffer = CausalAttentionWithoutBuffers(d_in, d_out, context_length, 0.0)
new_ca_without_buffer.load_state_dict(torch.load("model.pth"))

new_ca_without_buffer.mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])